In [2]:
import datetime
import os
import re
from collections import Counter
from datetime import datetime, timedelta

import numpy as np
from keras.preprocessing import sequence
import sys
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
import sklearn
from collections import Counter

from sklearn.metrics import pairwise_distances
import sys
sys.path.append('../')
from ftw_model.extract_feature import computing_feature_wo
from data_ori import load_dataset
import textdistance

/home/simon/miniconda3/envs/casas/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def read_hh_dataset(dataset_path):

    ann_dataset = pd.read_csv(dataset_path, sep='\t')

    raw_columns = ['Date & Time', 'Sensor ID', 'Room-level', 'Sensor location', 'Message', 'Sensor Type']
    ann_columns = raw_columns + ['Activity']

    ann_dataset.columns = ann_columns
    # ann_dataset['Activity'] = ann_dataset['Activity'].apply(lambda x: activity_mapping[x] if x in activity_mapping else x)

    ann_dataset['Date & Time'] = pd.to_datetime(ann_dataset['Date & Time'], format='%Y-%m-%d %H:%M:%S')
    start_time, end_time = ann_dataset['Date & Time'].min(), ann_dataset['Date & Time'].max()
    timeframed_dataset = ann_dataset.set_index(['Date & Time'])

    activity2id = {}
    count = 0
    for act in ann_dataset['Activity'].unique():
        if act != 'Other_Activity':
            activity2id[act] = count
            count += 1
    activity2id['Other_Activity'] = count
    
    return timeframed_dataset, start_time, end_time, activity2id

In [15]:
# load data in npy/cairo
X, Y, dictActivities = load_dataset('../dataset/cairo')
milan_X, milan_Y, milan_dictActivities = load_dataset('../dataset/milan')

In [4]:
X, Y, dictActivities = np.load('../npy/cairo-x.npy', allow_pickle=True), np.load('../npy/cairo-y.npy', allow_pickle=True), np.load('../npy/cairo-labels.npy', allow_pickle=True).item()
milan_X, milan_Y, milan_dictActivities = np.load('../npy/milan-x.npy', allow_pickle=True), np.load('../npy/milan-y.npy', allow_pickle=True), np.load('../npy/milan-labels.npy', allow_pickle=True).item()

In [5]:
dictActivities

{'Other': 0,
 'Bed_to_toilet': 6,
 'Take_medicine': 2,
 'Eat': 5,
 'Leave_Home': 4,
 'Sleep': 3,
 'Work': 1}

In [6]:
milan_dictActivities

{'Other': 0,
 'Bed_to_toilet': 8,
 'Sleep': 3,
 'Take_medicine': 2,
 'Relax': 4,
 'Cook': 7,
 'Work': 1,
 'Leave_Home': 5,
 'Bathing': 9,
 'Eat': 6}